In [14]:
# -=-<[ Bismillahirrahmanirrahim ]>-=-
# -*- coding: utf-8 -*-
# @Date    : 2024-10-15 17:07:17
# @Author  : Dahir Muhammad Dahir (dahirmuhammad3@gmail.com)
# @Link    : link
# @Version : 1.0.0


from sqlalchemy import create_engine
from sqlalchemy.orm import DeclarativeBase
from sqlalchemy.orm import Mapped
from sqlalchemy.orm import mapped_column
from sqlalchemy.orm import relationship
from sqlalchemy import String, ForeignKey

engine = create_engine("sqlite+pysqlite:///:memory:", echo=True)



In [17]:

class Base(DeclarativeBase):
    pass

Base.metadata
Base.registry

In [18]:

class User(Base):
    __tablename__ = "user_account"

    id: Mapped[int] = mapped_column(primary_key=True)
    name: Mapped[str] = mapped_column(String(30))
    fullname: Mapped[str | None]

    addresses: Mapped[list["Address"]] = relationship(back_populates="user")

    def __repr__(self) -> str:
        return f"User(id={self.id!r}, name={self.name!r}, fullname={self.fullname!r})"


class Address(Base):
    __tablename__ = "address"

    id: Mapped[int] = mapped_column(primary_key=True)
    email_address: Mapped[str]
    user_id: Mapped[int] = mapped_column(ForeignKey("user_account.id"))

    user: Mapped[User] = relationship("User", back_populates="addresses")

    def __repr__(self) -> str:
        return f"Address(id={self.id!r}, email_address={self.email_address!r})"



In [ ]:
# create the tables in db

Base.metadata.create_all(engine)

In [20]:
# inserting data using core
from sqlalchemy import insert

stmt = insert(User).values(name="dahir", fullname="dahir muhammad")
print(stmt)


INSERT INTO user_account (name, fullname) VALUES (:name, :fullname)


In [21]:
with engine.connect() as conn:
    result = conn.execute(stmt)
    conn.commit()


2024-10-15 19:20:17,999 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-10-15 19:20:18,000 INFO sqlalchemy.engine.Engine INSERT INTO user_account (name, fullname) VALUES (?, ?)
2024-10-15 19:20:18,000 INFO sqlalchemy.engine.Engine [generated in 0.00113s] ('dahir', 'dahir muhammad')
2024-10-15 19:20:18,001 INFO sqlalchemy.engine.Engine COMMIT


In [22]:
result.inserted_primary_key

(1,)